In [36]:
# Import necessary libraries
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler,StandardScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score

In [37]:
data = pd.read_csv('Super_Store_preprocessed.csv', encoding='latin1')
data.columns

Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_name',
       'segment', 'state', 'country', 'market', 'region', 'product_id',
       'category', 'sub_category', 'product_name', 'sales', 'quantity',
       'discount', 'profit', 'shipping_cost', 'order_priority', 'year',
       'Return'],
      dtype='object')

In [38]:
# Separate features and target
X = data.drop(['discount', 'order_date', 'product_name', 'order_id', 'ship_date', 'customer_name','product_id','country'], axis=1)
y = data['discount']

In [19]:
X.head()

,ship_mode,segment,state,market,region,category,sub_category,sales,quantity,discount,shipping_cost,order_priority,year,Return
0,Standard Class,Consumer,Constantine,Africa,Africa,Office Supplies,Storage,408,2,0.0,35.46,Medium,2011,0.000
1,Standard Class,Consumer,New South Wales,APAC,Oceania,Office Supplies,Supplies,120,3,0.1,9.72,Medium,2011,0.000
2,Second Class,Consumer,Budapest,EMEA,EMEA,Office Supplies,Storage,66,4,0.0,8.17,High,2011,0.000
3,Second Class,Home Office,Stockholm,EU,North,Office Supplies,Paper,45,3,0.5,4.82,High,2011,-26.055
4,Standard Class,Consumer,New South Wales,APAC,Oceania,Furniture,Furnishings,114,5,0.1,4.70,Medium,2011,0.000


In [20]:
y

0        106.1400
1         36.0360
2         29.6400
3          0.0000
4         37.7700
           ...   
51285      4.5188
51286      0.4200
51287     12.3600
51288      0.5600
51289      0.0000
Name: profit, Length: 51290, dtype: float64

In [39]:


# Automatically select categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['number']).columns

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the preprocessing for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(objective='reg:squarederror', random_state=42))])

param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__learning_rate': [0.05, 0.1, 0.5],
    'model__max_depth': [3, 5, 7],
}



# Setup GridSearchCV
grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

# Train the model using GridSearchCV
grid_search.fit(X_train, y_train)

# Evaluate the model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)




In [40]:
# Calculate evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"ROOt Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Best Parameters: {'model__learning_rate': 0.5, 'model__max_depth': 7, 'model__n_estimators': 200}
ROOt Mean Squared Error: 0.0017730420268778665
R^2 Score: 0.9610692383579355


In [41]:
f"The y_trian mean { y_train.mean()}" 

'The y_trian mean 0.14296315071163967'


# Model Evaluation Conclusion

The model was optimized with the following best parameters:
- **learning_rate** = 0.5
- **max_depth** = 7
- **n_estimators** = 200

## Performance Metrics
- **Root Mean Squared Error (RMSE)**: 0.00177
- **R² Score**: 0.9611
- **Mean of y_train**: 0.14296

### Interpretation:
- The **RMSE** of 0.00177 is extremely low, suggesting that the model's predictions are highly accurate, with minimal deviation from the actual values of the "discount" variable.
- The **R² Score** of 0.9611 means that the model explains **96.1%** of the variance in the "discount" values. This indicates that the model captures almost all of the underlying patterns in the data.
- The mean value of `y_train` is 0.14296, which further emphasizes how small the error (RMSE) is in comparison to the target variable's scale. The average error is much smaller than the mean of the "discount," suggesting a strong predictive performance.

## Conclusion:
The model performs exceptionally well, providing highly accurate predictions of the "discount" based on the data. With a high R² score and a very low RMSE, the model is both effective in explaining the variance and minimizing the prediction error. This suggests that the model is well-suited for predicting the "discount" variable with minimal room for improvement.
